In [25]:
import os
import importlib
import logging
import time
importlib.reload(logging)
import framework
importlib.reload(framework)
import bert_qa
importlib.reload(bert_qa)
import infer_bert_qa
importlib.reload(infer_bert_qa)
import bert_utils
importlib.reload(bert_utils)
import pandas as pd
from framework import DataCuration, FeatureEngineering
from bert_qa import TaskQA, FeatureEngineeringQA, BERTQA

# Define some constants and configurations
logging.getLogger().setLevel(logging.INFO)
ACCESS_TOKEN = 'WUpGevbWC9lsnTW8quNUtmWRdAEM89'

Set up the task details. This notebook handles Question Answering for CARTA dataset.

Example

context = "New Zealand (Māori: Aotearoa) is a sovereign island country in the southwestern Pacific Ocean. It has a total land area of 268,000 square kilometres (103,500 sq mi), and a population of 4.9 million. New Zealand's capital city is Wellington, and its most populous city is Auckland."

questions = "How many people live in New Zealand?", "What's the largest city?"


In [26]:
DATASET = 'carta' # supports w2 and resume
TASK_CONFIG = {
    'task': 'qa'
}

task = TaskQA(TASK_CONFIG)

Set paths for datasets and goldens (local or ib, both work).
Specify configurations

In [27]:
CARTA_DATA = [
   '/Users/ahsaasbajaj/Documents/Data/QA_model/data'
]

DATASET_CONFIG = {
    'path': CARTA_DATA,
    'is_local': True, 
    'file_type': 'ibocr',
    'identifier': lambda path: os.path.basename(path).split('.ibocr')[0],
    'convert2txt': True
}

CARTA_GOLDEN = None
GOLDEN_CONFIG = None

data = DataCuration(ACCESS_TOKEN, DATASET_CONFIG, GOLDEN_CONFIG)

INFO:root:Loading dataset from /Users/ahsaasbajaj/Documents/Data/QA_model/data
INFO:root:1 files loaded


In [28]:
data.dataset

{'annotated_AOI_4.pdf': <instabase.ocr.client.libs.ibocr.ParsedIBOCR at 0x153671a20>}

In [29]:
# open_queries = [ 
#                 "Who is incorporating the company?",
#                 "How many shares are being created?",
#                 "What are the number of authorized shares?",
#                 "What are the Preferred stocks?",
#                 "What are the Non-cumulative dividends?",
#                 "What are the Common stocks?",
#                 "What is the Dividend rate per annum per preferred share type?",
#                 "What is the original issue price per share?",
#                 "What is the seniority of preferred share?",
#                 "What is the liquidation preference?",
#                 "What is the conversion price"
#                 ]

In [30]:
query = "What are the Preferred stocks?"

NUM_FILES = len(data.dataset.keys())
stime = time.time()

DATA_ARGS = {
    'task': task,
    'dataset': data
}

TRAINING_ARGS = {
'model_file_or_path': "/Users/ahsaasbajaj/Documents/Data/QA_model/model", # finetuned checkpoint available directly
'gpu': False,
'output_dir': '../outputs/bert_qa'
}

model = BERTQA(DATA_ARGS, TRAINING_ARGS)
output = model.predict(query)

etime = time.time()
logging.info('Total time {} seconds'.format(etime - stime))

INFO:root: Total number of Files: 1
INFO:root:File name: annotated_AOI_4.pdf
convert squad examples to features:   0%|          | 0/1 [00:00<?, ?it/s]Process ForkPoolWorker-4:
Traceback (most recent call last):
  File "/Users/ahsaasbajaj/.pyenv/versions/3.7.3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/ahsaasbajaj/.pyenv/versions/3.7.3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/ahsaasbajaj/.pyenv/versions/3.7.3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/Users/ahsaasbajaj/.pyenv/versions/3.7.3/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/Users/ahsaasbajaj/.pyenv/versions/3.7.3/lib/python3.7/site-packages/transformers/data/processors/squad.py", line 185, in squad_convert_example_to_features
    is_max_context = _new_check_is_max_context(spans,

KeyboardInterrupt: 

In [24]:
output = output.set_index('filename')
filenames = output.index.to_list()

for filename in filenames:
    print("filename: ", filename)

    for col in output.columns.to_list():
        print("query: ", col)
        answer = output.loc[filename, col]
        print("answer: ", answer)
    

filename:  annotated_AOI_4.pdf
query:  What are the Preferred stocks?
answer:  Series A Preferred Stock and Series A-1 Preferred Stock
